## Segmenting and Clustering Neighborhoods in Toronto part :3

In [1]:
import numpy as np 
import pandas as pd
import requests 
from pandas.io.json import json_normalize 
import json 
from geopy.geocoders import Nominatim 
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium

In [2]:
df_neigh= pd.read_csv('C:\\Users\\Ashkan\\DS_training\\Final Capstone_IBM\\df_Toronto2.csv')
df_neigh.drop('Unnamed: 0',axis=1,inplace=True)
df_neigh.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [3]:
print('This dataframe has {} boroughs and {} neighborhoods.'.format(len(df_neigh['Borough'].unique()),df_neigh.shape[0])
)

This dataframe has 10 boroughs and 103 neighborhoods.


In [4]:
address = 'Toronto, Canada'
geolocator = Nominatim(user_agent="can_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


#### Map creating of Toronto with visualizing the neighbourhoods:

In [5]:
toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
for lat, lng, borough, neighborhood in zip(df_neigh['Latitude'], df_neigh['Longitude'], df_neigh['Borough'], df_neigh['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color='green',
        fill_opacity=0.8,
        parse_html=False).add_to(toronto)  
    
toronto

#### Using the Foursquare API to explore the neighborhoods:

In [6]:
LIMIT = 70
CLIENT_ID = 'GNRHQWT3CFCASPET1ZFGXS44E3QFY0PIZMLUMQ2D4C5KRFQY' 
CLIENT_SECRET = 'RWC1WXVECHQYDYLQGZ1MVBJ15SAUSGLWBL50EGTZ3JTPKCSX' 
VERSION = '20180605' 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: GNRHQWT3CFCASPET1ZFGXS44E3QFY0PIZMLUMQ2D4C5KRFQY
CLIENT_SECRET:RWC1WXVECHQYDYLQGZ1MVBJ15SAUSGLWBL50EGTZ3JTPKCSX


In [7]:
def neighbor(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
       
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

##### Use the neighbor function on each neighborhood and create a new dataframe called venues_Toronto :

In [8]:
venues_Toronto = neighbor(names=df_neigh['Neighborhood'],
                                   latitudes=df_neigh['Latitude'],
                                   longitudes=df_neigh['Longitude'],
                                  )

Rouge,Malvern
Highland Creek,Rouge Hill,Port Union
Guildwood,Morningside,West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park,Ionview,Kennedy Park
Clairlea,Golden Mile,Oakridge
Cliffcrest,Cliffside,Scarborough Village West
Birch Cliff,Cliffside West
Dorset Park,Scarborough Town Centre,Wexford Heights
Maryvale,Wexford
Agincourt
Clarks Corners,Sullivan,Tam O'Shanter
Agincourt North,L'Amoreaux East,Milliken,Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview,Henry Farm,Oriole
Bayview Village
Silver Hills,York Mills
Newtonbrook,Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park,Don Mills South
Bathurst Manor,Downsview North,Wilson Heights
Northwood Park,York University
CFB Toronto,Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens,Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West,Riverdale
The Beaches West,Indi

In [9]:
venues_Toronto.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge,Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Chris Effects Painting,43.784343,-79.163742,Construction & Landscaping
2,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Guildwood,Morningside,West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Marina Spa,43.766000,-79.191000,Spa


In [10]:
venues_Toronto.shape

(1949, 7)

####  Analyzing  Neighborhoods:

In [11]:
trnt = pd.get_dummies(venues_Toronto[['Venue Category']], prefix="", prefix_sep="")
trnt['Neighborhood'] = venues_Toronto['Neighborhood'] 
fixed_columns = [trnt.columns[-1]] + list(trnt.columns[:-1])
trnt = trnt[fixed_columns]
trnt.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
trnt.shape

(1949, 258)

####  grouping rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [13]:
trnt_gp = trnt.groupby('Neighborhood').mean().reset_index()
trnt_gp.head()

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,"Adelaide,King,Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.028571,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.090909,0.0,0.0,0.0,0.0,0.0,0.0
4,"Alderwood,Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
trnt_gp.shape

(99, 258)

####  neighborhoods along with the top 7 most common venues:

In [15]:
num_top_venues = 7

for hood in trnt_gp['Neighborhood']:
    print("----"+hood+"----")
    temp = trnt_gp[trnt_gp['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
                venue  freq
0         Coffee Shop  0.06
1          Steakhouse  0.04
2                Café  0.04
3          Restaurant  0.04
4     Thai Restaurant  0.03
5         Pizza Place  0.03
6  Seafood Restaurant  0.03


----Agincourt----
                       venue  freq
0  Latin American Restaurant  0.25
1               Skating Rink  0.25
2                     Lounge  0.25
3             Breakfast Spot  0.25
4         Light Rail Station  0.00
5              Luggage Store  0.00
6         Mac & Cheese Joint  0.00


----Agincourt North,L'Amoreaux East,Milliken,Steeles East----
                       venue  freq
0                       Park  0.33
1                Coffee Shop  0.33
2                 Playground  0.33
3             Medical Center  0.00
4  Middle Eastern Restaurant  0.00
5         Mexican Restaurant  0.00
6              Metro Station  0.00


----Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,This

In [16]:
def common_venus(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [17]:
num_top_venues = 7

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = trnt_gp['Neighborhood']

for ind in np.arange(trnt_gp.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = common_venus(trnt_gp.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Restaurant,Steakhouse,Café,American Restaurant,Asian Restaurant,Seafood Restaurant
1,Agincourt,Skating Rink,Latin American Restaurant,Lounge,Breakfast Spot,Dance Studio,Drugstore,Donut Shop
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Park,Playground,Coffee Shop,Gay Bar,Creperie,Doner Restaurant,Dog Run
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Pizza Place,Discount Store,Beer Store,Video Store,Japanese Restaurant,Pharmacy
4,"Alderwood,Long Branch",Pizza Place,Skating Rink,Gym,Pharmacy,Pool,Pub,Dance Studio


####  Neighborhoods Clustering :
use k-means to cluster the neighborhood into 7 clusters.

In [18]:
K =7
trnt_cluster = trnt_gp.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=K, random_state=0).fit(trnt_cluster)
kmeans.labels_[0:10] 

array([2, 2, 0, 2, 2, 2, 2, 2, 2, 2])